# About

This notebook will introduce the `OPTIONAL MATCH` and `ORDER BY` cypher keywords.

In [ ]:
from neo4j import GraphDatabase, Record, ResultSummary, EagerResult
from neo4j.time import Date

import pandas as pd
pd.set_option('display.max_colwidth', 100)

import os 
import sys
from dotenv import load_dotenv 
load_dotenv()

# Add the utils directory to sys.path
sys.path.append(os.path.abspath("../utils"))

from Neo4jParser import Neo4jParser


NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

## `OPTIONAL MATCH`

* `OPTIONAL MATCH` is great because it can be used to determine if something exists, but it is not required so it will not break the entire query.
* It is better to use `MATCH` since the query writer has more control over exactly what is being asked. In downstream data processing, a data element that may or may not exist could be difficult to handle.
* `OPTIONAL MATCH` is a great utility for creating relationships. If you are batch uploading records, it is not efficient to define each query. Rather you can write your query to create a relationship under certain conditions and `OPTIONAL MATCH` is great for this scenario. If the condition doesn't exist to create a relationship, no worries.

In [3]:
# Let's say we want to return all actors in a movie, and optionally the director if it's provided
result = driver.execute_query(
    """ 
    MATCH (movie:Movie {title: "Joker"})-[:ACTED_IN]-(actors:Person)
    OPTIONAL MATCH (movie)-[:DIRECTED]-(director:Person)
    RETURN DISTINCT movie.title AS movie, COUNT(actors) AS number_of_actors, director.name AS director
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 1 records after 4 ms and completed after 7 ms.

Query executed against database: 'neo4j':  
    MATCH (movie:Movie {title: "Joker"})-[:ACTED_IN]-(actors:Person)
    OPTIONAL MATCH (movie)-[:DIRECTED]-(director:Person)
    RETURN DISTINCT movie.title AS movie, COUNT(actors) AS number_of_actors, director.name AS director
    


{'movie': ['Joker'], 'director': [None], 'number_of_actors': [3]}

**NOTE:** Above, notice the query returned the movie, the number of actors, and the director even though a director doesn't exist in the graph. If we were to remove the 'OPTIONAL' keyword in `OPTIONAL MATCH` the query would return nothing, since the pattern or condition provided does not exist.

In [4]:
# Same query but remove 'OPTIONAL' from 'OPTIONAL MATCH'
result = driver.execute_query(
    """ 
    MATCH (movie:Movie {title: "Joker"})-[:ACTED_IN]-(actors:Person)
    MATCH (movie)-[:DIRECTED]-(director:Person)
    RETURN DISTINCT movie.title AS movie, COUNT(actors) AS number_of_actors, director.name AS director
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 0 records after 0 ms and completed after 0 ms.

Query executed against database: 'neo4j':  
    MATCH (movie:Movie {title: "Joker"})-[:ACTED_IN]-(actors:Person)
    MATCH (movie)-[:DIRECTED]-(director:Person)
    RETURN DISTINCT movie.title AS movie, COUNT(actors) AS number_of_actors, director.name AS director
    


{}

In [5]:
# Same query but remove 'OPTIONAL' from 'OPTIONAL MATCH'
result = driver.execute_query(
    """ 
    MATCH (movie:Movie {title: "As Good as It Gets"})-[:ACTED_IN]-(actors:Person)
    MATCH (movie)-[:DIRECTED]-(director:Person)
    RETURN DISTINCT movie.title AS movie, COUNT(actors) AS number_of_actors, director.name AS director
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)
data

Started streaming 1 records after 0 ms and completed after 0 ms.

Query executed against database: 'neo4j':  
    MATCH (movie:Movie {title: "As Good as It Gets"})-[:ACTED_IN]-(actors:Person)
    MATCH (movie)-[:DIRECTED]-(director:Person)
    RETURN DISTINCT movie.title AS movie, COUNT(actors) AS number_of_actors, director.name AS director
    


{'movie': ['As Good as It Gets'],
 'director': ['James L. Brooks'],
 'number_of_actors': [4]}

In [6]:
# Use optional match to create a new "FRIENDS_WITH" relationship between actors and directors of a movie
result = driver.execute_query(
    """ 
    MATCH (movie:Movie)
    OPTIONAL MATCH (actor:Person)-[:ACTED_IN]->(movie)
    OPTIONAL MATCH (director:Person)-[:DIRECTED]->(movie)
    WITH actor, director, movie
    WHERE actor IS NOT NULL AND director IS NOT NULL
    MERGE (actor)-[:FRIENDS_WITH]->(director)
    RETURN actor.name, director.name, movie.title
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, False)


Started streaming 200 records after 0 ms and completed after 9 ms.

Query executed against database: 'neo4j':  
    MATCH (movie:Movie)
    OPTIONAL MATCH (actor:Person)-[:ACTED_IN]->(movie)
    OPTIONAL MATCH (director:Person)-[:DIRECTED]->(movie)
    WITH actor, director, movie
    WHERE actor IS NOT NULL AND director IS NOT NULL
    MERGE (actor)-[:FRIENDS_WITH]->(director)
    RETURN actor.name, director.name, movie.title
    


## `ORDER BY`

* `ORDER BY` is a way to sort results from a query, just like how it is used in SQL.
* `ORDER BY` can be used with `ASC` for 'ascending' and `DESC` or descending.
    * For example, `... RETURN p.name, p.title ORDER BY p.name`. This will sort 'p.name' in ascending order.
    * For example, `... RETURN p.name, COUNT(*) ORDER BY COUNT(*) DESC`. This will sort the count of 'p.name' descending.
* Sorting by more than one object happens sequentially, in order.

In [ ]:
# Return the number of people for each movie and sort by the count descending
result = driver.execute_query(
    """ 
    MATCH (movie:Movie)--(person:Person)
    RETURN movie.title, COUNT(*) AS num_people ORDER BY num_people DESC
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, True)
data.head()

Started streaming 40 records after 33 ms and completed after 33 ms.

Query executed against database: 'neo4j':  
    MATCH (movie:Movie)--(person:Person)
    RETURN movie.title, COUNT(*) AS num_people ORDER BY num_people DESC
    


,num_people,movie.title
0,14,A Few Good Men
1,12,Jerry Maguire
2,12,Speed Racer
3,11,V for Vendetta
4,9,The Green Mile


In [12]:
# Sort by two columns
result = driver.execute_query(
    """ 
    MATCH (movie:Movie)--(person:Person)
    RETURN DISTINCT movie.title, person.name ORDER BY movie.title, person.name
    """,
    database_="neo4j"
)

data = Neo4jParser.parse(result, True, True)
data.head()

Started streaming 241 records after 42 ms and completed after 42 ms.

Query executed against database: 'neo4j':  
    MATCH (movie:Movie)--(person:Person)
    RETURN DISTINCT movie.title, person.name ORDER BY movie.title, person.name
    


,movie.title,person.name
0,A Few Good Men,Aaron Sorkin
1,A Few Good Men,Christopher Guest
2,A Few Good Men,Cuba Gooding Jr.
3,A Few Good Men,Demi Moore
4,A Few Good Men,J.T. Walsh
